In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Goal: based on daily prices, calculate yearly return, average daily return, daily volatility, annualized daily vol


tickers = [
    'TUPRS.IS', 
    'TCELL.IS', 
    'CCOLA.IS', 
    'SISE.IS', 
    'ENJSA.IS', 
    'ARCLK.IS', 
    'OTKAR.IS', 
    'SELEC.IS',
    'AKSA.IS',
    'DGNMO.IS'
]

start_date = '2016-01-01'  # YYYY-MM-DD
end_date = '2025-10-03'
interval = '1d'
filename = 'Turkish_10_companies.xlsx'


def calculate(tickers, start, end, interval):
    df = yf.download(tickers, start=start, end=end, interval=interval, auto_adjust=True)
    # when we first argument is list we download data for several stocks
    # as a result we get a table with multi-level columns, where level=0 (higher order) columns are Close, High, Low, Open, Volume
    # level=1 columns are tickers
    
    # Claculate daily returns for each stock as percentage change and append it original df to keep multi-column structure
    daily_returns = df['Close'].pct_change()
    daily_returns.columns = pd.MultiIndex.from_product([['Daily Return'], daily_returns.columns]) # create multi-column structure similar to df one
    df = pd.concat([df, daily_returns], axis=1) # label-alignment and appending from the right (label-alignment)

    # calculating yearly return. To avoid possible missing prices we check if a stock has prices at all at particular year (drop na values)
    # and in this shrinked series take first and last values available
    # first apply considers subdf as variable, second apply considers separate ticker series as variable .dropna() shrinks series to valid
    # number .size checks if this shrinked series has values at all

    year = df.index.year
    first_prices = df['Close'].groupby(year).apply(lambda subdf: subdf.apply(lambda ticker_series: ticker_series.dropna().iloc[0] if ticker_series.dropna().size else np.nan))
    last_prices = df['Close'].groupby(year).apply(lambda subdf: subdf.apply(lambda ticker_series: ticker_series.dropna().iloc[-1] if ticker_series.dropna().size else np.nan))
    return_yearly = last_prices/first_prices-1 # label and column alignment happens here
    # making multicolumn structure
    return_yearly.columns = pd.MultiIndex.from_product([['Yearly return'], return_yearly.columns])

    # now we calculate average yearly return and daily volatility of returns for each year

    daily_ret = df['Close'].pct_change()
    avg_daily = daily_ret.groupby(year).mean()
    avg_daily.columns = pd.MultiIndex.from_product([['Average daily return'], avg_daily.columns])
    vol_daily = daily_ret.groupby(year).std(ddof=1)
    vol_daily.columns = pd.MultiIndex.from_product([['Daily volatility'], vol_daily.columns])
    
    # we calculate yearly volatility as daily standard deviation of returns multiplied by square root of N:
    # intuition: daily returns r(t) are assumed to be independent and indentically distributed with same mean and standard deviation. Hence 
    # Var(sum(r(t)) = Var(r(1)) + ...Var(r(t)) = N * (std.dev)^2. Hence yearly volatility is compute as sqrt(number of days in a year) * std.dev
    
    days_in_years = daily_ret.groupby(year).count()
    vol_yearly = vol_daily * np.sqrt(days_in_years)
    vol_yearly.rename(columns={'Daily volatility': 'Yearly volatility'}, inplace=True)
    df2 = pd.concat([return_yearly, avg_daily, vol_daily, vol_yearly], axis=1) # choose axis properly to perform concatenation we need
    
    with pd.ExcelWriter(filename) as writer:
        df.to_excel(writer, sheet_name='prices', na_rep='N/A')
        df2.to_excel(writer, sheet_name='calculated_indicators', na_rep='N/A')

result_df = calculate(tickers, start_date, end_date, interval)

In [11]:
# Turkey debt to gdp and budget deficit to GDP from 2016 to 2025 (partial data available)
# Source: IMF

import requests
import numpy as np
import pandas as pd

endpoint = f'https://www.imf.org/external/datamapper/api/v1/'

tickers = {
    'debt_to_gdp': 'GGXWDG_NGDP',
    'budget_to_gdp': 'GGXCNL_NGDP'
}
country_code = 'TUR'
years_number = 10

years = ','.join([str(2016 + i) for i in range(years_number+1)])


debt_link = f"{endpoint}{tickers['debt_to_gdp']}/{country_code}"
budget_link = f"{endpoint}{tickers['budget_to_gdp']}/{country_code}"

params = {'periods': years}

# Requesting debt-to-gdp data:
resp = requests.get(debt_link, params=params)
debt_data = resp.json()
debt_dict = debt_data['values'][tickers['debt_to_gdp']][country_code]

# Requesting budget-to-gdp data:
resp = requests.get(budget_link, params=params)
budget_data = resp.json()
budget_dict = budget_data['values'][tickers['budget_to_gdp']][country_code]

# Constructing dataframes:

debt_df = pd.DataFrame.from_dict(debt_dict, orient='index', columns = ['debt_to_gdp'])
debt_df.index = pd.to_datetime(debt_df.index, format='%Y')

budget_df = pd.DataFrame.from_dict(budget_dict, orient='index', columns=['budget_deficit_to_gdp'])
budget_df.index = pd.to_datetime(budget_df.index, format='%Y')


# As the lowest frequency for this type of stock data is one year,
# we interpolate to months using interpolated method and method type 'time'

debt_df = debt_df.resample('MS').asfreq()
budget_df = budget_df.resample('MS').asfreq()

#interpolating

debt_df['debt_to_gdp'] = debt_df['debt_to_gdp'].interpolate(method='time')
budget_df['budget_deficit_to_gdp'] = budget_df['budget_deficit_to_gdp'].interpolate(method='time')

with pd.ExcelWriter('DebtAndBudgetDef-to-GDP.xlsx') as writer:
    debt_df.to_excel(writer, sheet_name='Debt_to_GDP', na_rep='NA')
    budget_df.to_excel(writer, sheet_name='BudgetDef_to_GDP', na_rep='NA')
    



In [3]:
#USDTRY monthly returns

import yfinance as yf
import pandas as pd
import numpy as np

usdtry = yf.download('TRY=X', start='2016-01-01', end='2025-10-01', interval='1mo')
usdtry['monthly return'] = usdtry['Close']/usdtry['Close'].shift(1) - 1

usdtry.to_excel('USDTRY Monthly Return.xlsx')

/var/folders/4n/k1h49sjj70j9w9yg9yc94v_c0000gn/T/ipykernel_16437/275060060.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  usdtry = yf.download('TRY=X', start='2016-01-01', end='2025-10-01', interval='1mo')
[*********************100%***********************]  1 of 1 completed
